# Trabalhar com dados numéricos

No notebook anterior, treinamos um modelo de k-nearest mais próximos em
adicionar dados.

No entanto, simplificamos demais o procedimento carregando um conjunto de dados que continha
dados exclusivamente numéricos. Além disso, usamos conjuntos de dados que já eram
dividido em conjuntos de teste de trem.

Neste bloco de notas, pretendemos:

* identificar dados numéricos em um conjunto de dados heterogêneo;
* selecionar o subconjunto de colunas correspondentes aos dados numéricos;
* usando um helper scikit-learn para separar os dados em conjuntos de teste de treinamento;
* treinar e avaliar um modelo mais complexo de scikit-learn.

Começaremos carregando o conjunto de dados do censo adulto usado durante os dados
exploração.

## Carregando todo o conjunto de dados

Como no bloco de notas anterior, contamos com o pandas para abrir o arquivo CSV em
para pandas dataframe.

In [2]:
import pandas as pd
 
adult_census = pd.read_csv("adult-census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns="education-num:")
adult_census.head()

,ID,age,workclass,fnlwgt:,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:,class
0,1,39,State-gov,77516,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,2,50,Self-emp-not-inc,83311,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,3,38,Private,215646,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,4,53,Private,234721,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,5,28,Private,338409,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
adult_census = adult_census.drop(columns=['ID','fnlwgt:'])
adult_census.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:,class
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K



A próxima etapa separa o destino dos dados. Executamos o mesmo
procedimento no caderno anterior.

In [5]:
data, target = adult_census.drop(columns="class"), adult_census["class"]

In [6]:
data.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [7]:
target

0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
32556    <=50K
32557     >50K
32558    <=50K
32559    <=50K
32560     >50K
Name: class, Length: 32561, dtype: object

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">Here and later, we use the name <tt class="docutils literal">data</tt> and <tt class="docutils literal">target</tt> to be explicit. In
scikit-learn documentation, <tt class="docutils literal">data</tt> is commonly named <tt class="docutils literal">X</tt> and <tt class="docutils literal">target</tt> is
commonly called <tt class="docutils literal">y</tt>.</p>
</div>

Neste ponto, podemos nos concentrar nos dados que queremos usar para treinar nosso
modelo preditivo.

## Identificar dados numéricos

Os dados numéricos são representados por números. Eles estão ligados a mensuráveis
dados (quantitativos), como idade ou o número de horas que uma pessoa trabalha um
semana.

Os modelos preditivos são projetados nativamente para trabalhar com dados numéricos.
Além disso, os dados numéricos geralmente requerem muito pouco trabalho antes de obter
começou com o treinamento.

A primeira tarefa aqui será identificar dados numéricos em nosso conjunto de dados.

<div class="admonition caution alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Caution!</p>
<p class="last">Os dados numéricos são representados com números, mas os números nem sempre são
representando dados numéricos. As categorias já podem ser codificadas com
números e você precisará identificar esses recursos.</p>
</div>

Assim, podemos verificar o tipo de dados para cada uma das colunas no conjunto de dados.

In [8]:
data.dtypes

age                 int64
workclass          object
education:         object
marital-status:    object
occupation:        object
relationship:      object
race:              object
sex:               object
capital-gain:       int64
capital-loss:       int64
hours-per-week:     int64
native-country:    object
dtype: object

Parece que temos apenas dois tipos de dados. Podemos ter certeza, verificando o exclusivo
tipos de dados.

In [9]:
data.dtypes.unique()

array([dtype('int64'), dtype('O')], dtype=object)

Na verdade, os únicos dois tipos no conjunto de dados são inteiro e objeto.
Podemos olhar as primeiras linhas do dataframe para entender o
significado do tipo de dados `object`.

In [10]:
data.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


Vemos que o tipo de dados `object` corresponde a colunas contendo strings.
Como vimos na seção de exploração, essas colunas contêm categorias e nós
veremos mais tarde como lidar com isso. Podemos selecionar as colunas que contêm
inteiros e verifique seu conteúdo.

In [11]:
numerical_columns = ["age", "capital-gain:", "capital-loss:", "hours-per-week:"]
data[numerical_columns].head()

,age,capital-gain:,capital-loss:,hours-per-week:
0,39,2174,0,40
1,50,0,0,13
2,38,0,0,40
3,53,0,0,40
4,28,0,0,40


Agora que limitamos o conjunto de dados apenas a colunas numéricas,
podemos analisar esses números para descobrir o que eles representam. Podemos
identificar dois tipos de uso.

A primeira coluna, `"age"`, é autoexplicativa. Podemos notar que os valores
são contínuos, o que significa que podem ocupar qualquer número em um determinado intervalo. Vamos
descubra o que é esse intervalo:

In [12]:
data["age"].describe()

count    32561.000000
mean        38.581647
std         13.640433
min         17.000000
25%         28.000000
50%         37.000000
75%         48.000000
max         90.000000
Name: age, dtype: float64

Podemos perceber que a idade varia entre 17 e 90 anos.

Poderíamos estender nossa análise e descobriremos que `"capital-gain"`,
`"capital-loss"`, e `"hours-per-week"` também representam
dados.

Agora, armazenamos o subconjunto de colunas numéricas em um novo dataframe.

In [13]:
data_numeric = data[numerical_columns]

## Train-test split the dataset

No notebook anterior, carregamos dois conjuntos de dados separados: um de treinamento e
um teste. No entanto, ter conjuntos de dados separados em dois arquivos distintos é
incomum: na maioria das vezes, temos um único arquivo contendo todos os dados que
precisamos dividir uma vez carregado na memória.

Scikit-learn fornece a função auxiliar
`sklearn.model_selection.train_test_split` que é usado para automaticamente
dividir o conjunto de dados em dois subconjuntos.

In [14]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data_numeric, target, random_state=42, test_size=0.25)

<div class="admonition tip alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Tip</p>
<p class="last">Na configuração do scikit-learn, o parâmetro <tt class = "docutils literal"> random_state </tt> permite obter
resultados determinísticos quando usamos um gerador de números aleatórios. No
<tt class = "docutils literal"> train_test_split </tt> caso a aleatoriedade venha do embaralhamento dos dados, que
decide como o conjunto de dados é dividido em um trem e um conjunto de teste). </p>
</div>

Ao chamar a função `train_test_split`, especificamos que gostaríamos
ter 25% das amostras no conjunto de teste, enquanto as amostras restantes (75%)
estará disponível no conjunto de treinamento. Podemos verificar rapidamente se temos
o que esperávamos.

In [15]:
print(f"Number of samples in testing: {data_test.shape[0]} => "
      f"{data_test.shape[0] / data_numeric.shape[0] * 100:.1f}% of the"
      f" original set")

Number of samples in testing: 8141 => 25.0% of the original set


In [16]:
print(f"Number of samples in training: {data_train.shape[0]} => "
      f"{data_train.shape[0] / data_numeric.shape[0] * 100:.1f}% of the"
      f" original set")

Number of samples in training: 24420 => 75.0% of the original set


No notebook anterior, usamos um modelo de k-vizinhos mais próximos. Enquanto isso
modelo é intuitivo de entender, não é amplamente utilizado na prática. Agora nós
usará um modelo mais útil, chamado de regressão logística, que pertence ao
a família de modelos lineares.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p>In short, linear models find a set of weights to combine features linearly
and predict the target. For instance, the model can come up with a rule such
as:</p>
<ul class="simple">
<li>if <tt class="docutils literal">0.1 * age + 3.3 * <span class="pre">hours-per-week</span> - 15.1 &gt; 0</tt>, predict <tt class="docutils literal"><span class="pre">high-income</span></tt></li>
<li>otherwise predict <tt class="docutils literal"><span class="pre">low-income</span></tt></li>
</ul>
<p class="last">Linear models, and in particular the logistic regression, will be covered in
more details in the "Linear models" module later in this course. For now the
focus is to use this logistic regression model in scikit-learn rather than
understand how it works in details.</p>
</div>

Para criar um modelo de regressão logística no scikit-learn, você pode fazer:

In [17]:
# to display nice model diagram
from sklearn import set_config
set_config(display='diagram')

In [18]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

Agora que o modelo foi criado, você pode usá-lo exatamente da mesma maneira que
usamos o modelo de vizinhos k-mais próximos no notebook anterior. Dentro
em particular, podemos usar o método `fit` para treinar o modelo usando o
dados e rótulos:

In [19]:
model.fit(data_train, target_train)

LogisticRegression()

Também podemos usar o método `score` para verificar o desempenho estatístico do modelo
no conjunto de teste.

In [20]:
accuracy = model.score(data_test, target_test)
print(f"Accuracy of logistic regression: {accuracy:.3f}")

Accuracy of logistic regression: 0.801


Agora, a verdadeira questão é: este desempenho estatístico é relevante para um bom
modelo preditivo? Descubra resolvendo o próximo exercício!

Neste caderno, aprendemos a:

* identificar dados numéricos em um conjunto de dados heterogêneo;
* selecione o subconjunto de colunas correspondentes aos dados numéricos;
* use a função scikit-learn `train_test_split` para separar os dados em
  um trem e um conjunto de teste;
* treinar e avaliar um modelo de regressão logística.

# 📝 Exercício M1.03

O objetivo deste exercício é comparar o desempenho estatístico de nosso
classificador (81% de precisão) para alguns classificadores de linha de base que iriam ignorar o
inserir dados e, em vez disso, fazer previsões constantes.

- Qual seria a pontuação de um modelo que sempre prevê `'> 50K'`?
- Qual seria a pontuação de um modelo que sempre prevê `'<= 50K'`?
- A precisão de 81% ou 82% é uma boa pontuação para este problema?


Use um `DummyClassifier` e faça uma divisão de teste de trem para avaliar
sua precisão no conjunto de teste. Esta [link](https://scikit-learn.org/stable/modules/model_evaluation.html#dummy-estimators) mostra alguns exemplos de como avaliar o desempenho estatístico desses
modelos de linha de base.

Vamos primeiro dividir nosso conjunto de dados para ter o alvo separado dos dados
usado para treinar nosso modelo preditivo.

In [21]:
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=target_name)

Começamos selecionando apenas as colunas numéricas, como visto no anterior
caderno.

In [22]:
numerical_columns = [
    "age", "capital-gain:", "capital-loss:", "hours-per-week:"]

data_numeric = data[numerical_columns]

A seguir, vamos dividir os dados e o destino em um conjunto de treinamento e teste.

In [23]:
from sklearn.model_selection import train_test_split

data_numeric_train, data_numeric_test, target_train, target_test = \
    train_test_split(data_numeric, target, random_state=0)

Divida o conjunto de dados em conjuntos de treinamento e teste.

In [25]:
from sklearn.model_selection import train_test_split
# Write your code here.
data_train, data_test, target_train, target_test = train_test_split(
    data_numeric, target, random_state=42, test_size=0.25)

Use um `DummyClassifier` de forma que o classificador resultante sempre
prever a classe `'> 50K'`. Qual é a pontuação de precisão no conjunto de teste?
Repita o experimento sempre prevendo a classe `'<= 50K'`.

Dica: você pode consultar o parâmetro `strategy` do` DummyClassifier`
para alcançar o comportamento desejado.

In [27]:
from sklearn.dummy import DummyClassifier

class_to_predict = ">50K"
high_revenue_clf = DummyClassifier(strategy="constant",
                                   constant=class_to_predict)
high_revenue_clf.fit(data_numeric_train, target_train)
score = high_revenue_clf.score(data_numeric_test, target_test)
print(f"Accuracy of a model predicting only high revenue: {score:.3f}")

Accuracy of a model predicting only high revenue: 0.237


Vemos claramente que a pontuação está abaixo de 0,5, o que pode ser surpreendente à primeira vista. Vamos agora verificar o desempenho estatístico de um modelo que sempre prevê a classe de receita baixa, ou seja, "<= 50K".

In [28]:
class_to_predict = "<=50K"
low_revenue_clf = DummyClassifier(strategy="constant",
                                  constant=class_to_predict)
low_revenue_clf.fit(data_numeric_train, target_train)
score = low_revenue_clf.score(data_numeric_test, target_test)
print(f"Accuracy of a model predicting only low revenue: {score:.3f}")

Accuracy of a model predicting only low revenue: 0.763


Observamos que este modelo possui uma precisão superior a 0,5. Isso se deve ao fato de termos 3/4 da meta pertencendo à classe de baixa renda.

Portanto, qualquer modelo preditivo que forneça resultados abaixo desse classificador fictício não será útil.

In [29]:
adult_census["class"].value_counts()

<=50K    24720
>50K      7841
Name: class, dtype: int64

In [31]:
(target == "<=50K").mean()

0.7591904425539756

Na prática, poderíamos ter a estratégia "most_frequent" para prever a classe que mais aparece no alvo de treinamento.

In [32]:
most_freq_revenue_clf = DummyClassifier(strategy="most_frequent")
most_freq_revenue_clf.fit(data_numeric_train, target_train)
score = most_freq_revenue_clf.score(data_numeric_test, target_test)
print(f"Accuracy of a model predicting the most frequent class: {score:.3f}")

Accuracy of a model predicting the most frequent class: 0.763


Portanto, a precisão LogisticRegression (cerca de 81%) parece melhor do que a precisão DummyClassifier (cerca de 76%). De certa forma, é um pouco reconfortante, usar um modelo de aprendizado de máquina oferece um desempenho melhor do que sempre prever a turma da maioria, ou seja, a turma de baixa renda "<= 50 mil".